# 0. Libraries

In [1]:
import re
import pandas as pd
import numpy as np
import random
import seaborn as sns
import os
from scipy.stats import zscore
from scipy.stats import norm

import time

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact


from itertools import product

from sklearn.manifold import TSNE

In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Load and verify data, delete rows with missing 'Systeme_Etudie'  -> df_ini

In [3]:
def f_load(path,parameter):
    # This function allows us to load the data of one parameter
    # path: the location of the files
    # parameters: ['INHIB%', 'IC50', 'KI', 'nH', 'DELTA Fit', 'DELTA Inhib Obs', 'pIC50','pKi', 'DELTA Effet Obs', 'EC50', 'pEC50', '% Stim']

    # Read data
    df_ini= pd.read_csv(path+parameter+".csv")

    # Validate that the results are numbers and there are not special characters
    df_ini["Operateur_Resultat"]=df_ini["Operateur_Resultat"].astype(float)    

    # Delete if we don't have information about the name of studied system
    df_ini = df_ini[df_ini['Systeme_Etudie']!='Non renseigné']
    
    # Delete rows where Operateur_Resultat is nan
    df_ini=df_ini.dropna(subset=['Operateur_Resultat'])
    df_ini.reset_index(drop=True, inplace=True)

    # Change nan for '-' for string columns
    str_columns=['Domaine_therapeutique','Reference_Societe']
    df_ini[str_columns] = df_ini[str_columns].astype(str)
    df_ini[str_columns] = df_ini[str_columns].replace('nan','-')
    for icolumn in str_columns:
        df_ini.loc[:,icolumn] = df_ini[icolumn].str.replace('.0', '')
    
    operateur_resultat = df_ini['Operateur_Resultat']
    return df_ini,operateur_resultat

# 2. Small pause for analysis

In [4]:
def fv_data_analysis(operateur_resultat):
        # Create Histogram for the distribution of Operateur_Resultat
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # First histogram
        axes[0].hist(operateur_resultat) #, bins=200, edgecolor='black')
        axes[0].set_title('Histogram of Operateur_Resultat (200 bins)')
        axes[0].set_xlabel('Operateur_Resultat')
        axes[0].set_ylabel('Frequency')
        
        # Second histogram
        axes[1].hist(operateur_resultat) #, bins=500, edgecolor='black')
        axes[1].set_title('Histogram of Operateur_Resultat (Zoom, 500 bins)')
        axes[1].set_xlabel('Operateur_Resultat')
        axes[1].set_ylabel('Frequency')
        axes[1].axis([-100, 200, 0, 50000])
        
        # Display the histograms
        plt.show()

In [5]:
def f_data_analysis(operateur_resultat): 
    # Custom statistics
    stats = operateur_resultat.describe()
    
    num_negative = (operateur_resultat < 0).sum()
    num_positive_over_100 = (operateur_resultat > 100).sum()
    p_num_negative =( (operateur_resultat < 0).sum()/stats['count'] )*100
    p_num_positive_over_100 = ( (operateur_resultat > 100).sum()/stats['count'] )*100
    
    min_value = operateur_resultat.min()
    max_value = operateur_resultat.max()
    
    statistics = {
        'count': stats['count'],
        'mean': stats['mean'],
        'std': stats['std'],
        'min': min_value,
        'max': max_value,
        'negatives': num_negative,
        'positives_over_100': num_positive_over_100,
        '% negatives': p_num_negative,
        '% positives_over_100': p_num_positive_over_100,
        '25%': stats['25%'],
        '50%': stats['50%'],
        '75%': stats['75%'],
    }
    
    rounded_data = {key: np.round(value, 2) if isinstance(value, (np.ndarray, float, int)) else value for key, value in statistics.items()}
    print(rounded_data)
    return rounded_data


## 2.1 Analysis of outliers

In [6]:
def find_outliers_percentile(operateur_resultat):
    Q1 = np.percentile(operateur_resultat, 10)
    Q3 = np.percentile(operateur_resultat, 90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers=[x for x in operateur_resultat if x < lower_bound or x > upper_bound]
    outliers.sort()
    return outliers

In [7]:
def find_outliers_zscore(operateur_resultat, threshold=6, percentage = 0.995 ):
    values = np.array(operateur_resultat)
    z_scores = zscore(operateur_resultat)  
    outliers = values[np.abs(z_scores) > threshold].tolist()
    outliers.sort()
    no_outliers = values[np.abs(z_scores) <= threshold].tolist()
    no_outliers.sort()
    return outliers, no_outliers

In [8]:
def calculate_mad(operateur_resultat):
    # Mean Absolute Deviation
    values = np.array(operateur_resultat)
    if len(values) == 0:
        return np.nan
    mad = np.mean(np.abs(values - np.mean(values)))  
    return mad

In [9]:
def f_find_outliers(operateur_resultat,label,visualization=0):
    # Group by Systeme_Etudie and aggregate Operateur_Resultat
    df_outliers = df_ini.groupby(label)['Operateur_Resultat'].agg(list)
    df_outliers = df_outliers.reset_index()
    
    # Calculate minimum, maximum, mean, outliers, and number of non-NaN elements for each list in Operateur_Resultat
    # df_outliers['not_nan'] = df_outliers['Operateur_Resultat'].apply(lambda x: sum(~np.isnan(x)))
    df_outliers['min'] = df_outliers['Operateur_Resultat'].apply(min)
    df_outliers['max'] = df_outliers['Operateur_Resultat'].apply(max)
    df_outliers['mean'] = df_outliers['Operateur_Resultat'].apply(lambda x: sum(x) / len(x))
    df_outliers['std_dev'] = df_outliers['Operateur_Resultat'].apply(lambda x: np.std(x) if len(x) > 0 else np.nan)
    # df_outliers['variance'] = df_outliers['Operateur_Resultat'].apply(lambda x: np.var(x, ddof=1) if len(x) > 0 else np.nan)
    df_outliers['mad'] = df_outliers['Operateur_Resultat'].apply(lambda x: np.mean(np.abs(x - np.mean(x))) if len(x) > 0 else np.nan)
    
    df_outliers['outliers_p'] = df_outliers['Operateur_Resultat'].apply(find_outliers_percentile)
    df_outliers[['outliers_z', 'non_outliers']] = df_outliers['Operateur_Resultat'].apply(
        lambda x: pd.Series(find_outliers_zscore(x))
    )
    
    df_outliers['num_outliers_p'] = df_outliers['outliers_p'].apply(len)
    df_outliers['num_outliers_z'] = df_outliers['outliers_z'].apply(len)
    
    # Comparing outlier by percentile vs outlier by z-score
    total_outliers_p = df_outliers['num_outliers_p'].sum()
    total_outliers_z = df_outliers['num_outliers_z'].sum()
    print(f'Total number of outliers_p: {total_outliers_p}')
    print(f'Total number of outliers_z: {total_outliers_z}')
    
    # Print the resulting DataFrame
    # print( df_outliers.drop(columns=['Operateur_Resultat','num_outliers_p','num_outliers_z' ]) )
    return df_outliers

In [10]:
def fv_outliers(df_outliers):
    # Visualization of outliers
    # Flatten outliers_p and outliers_z
    outliers_p_flat = [item for sublist in df_outliers['outliers_p'] for item in sublist]
    outliers_z_flat = [item for sublist in df_outliers['outliers_z'] for item in sublist]
   
    fig, axes = plt.subplots(2, 1, figsize=(10, 8))
    
    # Histogram of outliers_p
    axes[0].hist(outliers_p_flat, bins=200, edgecolor='black')
    axes[0].set_title('Histogram of outliers_p (Threshold: 99.5%)')
    axes[0].set_xlabel('Outlier Values')
    axes[0].set_ylabel('Frequency')
    axes[0].axis([-200, 200, 0, 380])
    
    # Histogram of outliers_z
    axes[1].hist(outliers_z_flat, bins=200, edgecolor='black')
    axes[1].set_title('Histogram of outliers_z (Threshold: 6z, 0.0027% )')
    axes[1].set_xlabel('Outlier Values')
    axes[1].set_ylabel('Frequency')
    axes[1].axis([-200, 200, 0, 380])
    
    plt.tight_layout()
    plt.show()

# 3. Two data analysis to answer two questions

In [11]:
def f_filtered(df_ini):
    # Delete outliers <-50 & values >150 according experts -> Analysis of signature
    invalid_results = df_ini[(df_ini['Operateur_Resultat'] > 150)] #(df_ini['Operateur_Resultat'] < -50) | 
    # invalid_produits = invalid_results['Produit_teste'].unique() # Use if we want to delete all the tests of the Product with outliers
    filtered_df = df_ini[~df_ini['Produit_teste'].isin(invalid_results)] # Delete only this test (one Systeme Etudie)
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [12]:
def f_fixed(df_ini):
    # This function fixes values according to toxic standards -> Analysis by toxicity
    tox_val=45
    # No toxic
    df_ini["Operateur_Resultat"][df_ini["Operateur_Resultat"]<tox_val]=0
    # Toxic
    df_ini["Operateur_Resultat"][df_ini["Operateur_Resultat"]>=tox_val]=1
    return df_ini

# 4. Differentiacite Product by Dose [df_ini -> df_dose]
Now also differenciate by EspeceSouche

We can diferenciate for other parameters doing the same

We will consider that a product 1 with dose 1 is different from product 1 with dose 2

Products to test:
*   S218939-1
*   S 18717-1

Etudes
*   010798/1;290798;92226



In [13]:
def f_drop_dup(df_add):
    # Differentiacite Product by Dose // We can diferenciate for other parameters doing the same
    # Explanation: We will consider that a product 1 with dose 1 is different from product 1 with dose 2
    # Products to test:[ 'S218939-1' , 'S 18717-1' ]
    # Etudes: 010798/1;290798;92226

    # Define the DataFrame columns
    group_cols = ['Produit','Systeme_Etudie']
    
    # 1. Differentiacite Product by Dose
    df_add["Produit"]=df_add["Produit_teste"].astype(str)+"_"+df_add["Dose_testee_Molar"].astype(str)
    
    # 2. If we found duplicated, we will chose by date -> The last one!
    # 2.1 Sort by date 
    df_add=df_add.sort_values(by='Date_Creation_Etude', ascending=False)

    # 2.2 Add all the context of 'Domaine_therapeutique' before eliminating duplicates    
    df_context = df_add.groupby(group_cols)['Domaine_therapeutique'].apply(lambda x: ' / '.join(sorted(set(x))) ).reset_index()
    df_add.drop(['Domaine_therapeutique'], axis=1,inplace=True)
    df_add = pd.merge(df_add, df_context, on=group_cols, how='left')
    
    # 2.3 Drop Duplicates
    df_add = df_add.drop_duplicates(subset=group_cols, keep='first')

    # 3. Only for visualization
    df_add=df_add.sort_values(by=['Produit'], ascending=True)
    df_add.reset_index(drop=True, inplace=True)

    # Organize columns
    df_add=df_add[['Produit', 'Type_de_protocole', 'Code_protocole_Numero_Version',  'Systeme_Etudie',
       'Reference_Societe', 'Titre_Abrege', 'Domaine_therapeutique',
       'Produit_teste', 'Dose_testee_Molar','Triplet_Teste', 'Serie_chimique', 
       'Nom_detude', 'Date_Creation_Etude', 'Parametre', 'Unite_du_Parametre',
       'Operateur_Resultat']]
    return df_add

In [14]:
def f_date(table):
    table['Date_Creation_Etude']=pd.to_datetime(table['Date_Creation_Etude'])
    filter_date = pd.to_datetime('01/10/2020')
    filtered_table = table[table['Date_Creation_Etude'] > filter_date].reset_index(drop=True)
    return filtered_table

In [15]:
def f_uniprot(path, df_group):
    df_uni= pd.read_csv(path+'Protein_OptimizedSafetyPanel_UniprotID'+".csv")
    df_uni['Reference_Societe']=df_uni['Reference_Societe'].astype(str)
    df_uniprot=df_group.merge(df_uni[['Reference_Societe','Uniprot Number']])   
    return df_uniprot

In [16]:
# Main
# 1. Load data
path=""
parameter='INHIB%'
label= 'Reference_Societe' #'Systeme_Etudie'
df_ini,operateur_resultat=f_load(path,parameter)

if label== 'Reference_Societe':
    unique_sys=open(path+"unique_sys_id.txt", "r").read().splitlines()
elif label== 'Systeme_Etudie':
    unique_sys=open(path+"unique_sys-txt.txt", "r").read().splitlines()

# 1.1. Small clean: Drop duplicates by Type de protocole: Sometimes we have the exact same information but it has been saved as ps and ops
df_ini=df_ini.sort_values(by=['Type_de_protocole' ])
columns_not_consider = df_ini.columns.difference(['Type_de_protocole','Domaine_therapeutique'])
df_ini = df_ini.drop_duplicates(subset=columns_not_consider, keep='first')

# 2. Small pause for analysis
# statistics=f_data_analysis(operateur_resultat)
# fv_data_analysis(operateur_resultat) 

# 2.1 Analysis of outliers
df_outliers=f_find_outliers(operateur_resultat,label)
# fv_outliers(df_outliers)

# 3. Differenciate product by dose by dose, delete duplicates 
df_group=f_drop_dup(df_ini)

# 4. Add UniProt code
df_uniprot=f_uniprot(path, df_group)

# 4. Two data analysis to answer two questions
df_filtered=f_filtered(df_uniprot) # Delete outliers
df_fixed=f_fixed(df_uniprot) # Use filtered as we delete outliers

# 5. Filter data coming from 1 October 2020 -> New Panel Safety
df_filtered_opt=f_date(df_filtered)
df_fixed_opt=f_date(df_fixed)

# Save all
name=path+"filtered.csv"
df_filtered.to_csv(name, index=False)
name=path+"fixed.csv"
df_fixed.to_csv(name, index=False)
name=path+"filtered_optimized.csv"
df_filtered_opt.to_csv(name, index=False)
name=path+"fixed_optimized.csv"
df_fixed_opt.to_csv(name, index=False)

Total number of outliers_p: 1026
Total number of outliers_z: 93


In [19]:
df_fixed_opt

,Produit,Type_de_protocole,Code_protocole_Numero_Version,Systeme_Etudie,Reference_Societe,Titre_Abrege,Domaine_therapeutique,Produit_teste,Dose_testee_Molar,Triplet_Teste,Serie_chimique,Nom_detude,Date_Creation_Etude,Parametre,Unite_du_Parametre,Operateur_Resultat,Uniprot Number
0,AGI-0074986_1E-05M,Optimized Security Profile,BY84EXT - 3,UP-NORA,355,Transporteur NA(h)/[3H] Nisoxétine/CHO,Oncology,AGI-0074986,1E-05M,AGI-0074986_03_AGIOS_AGI-0074986,NaN,EASI_0002295371,2021-09-27,INHIB%,%,0.0,P23975
1,AGI-0074986_1E-05M,Security Profile,BX41EXT - 3,CANAL CA2+ TYP L * DIHYDROPYRIDINE SITE,161,CA2+ TYP L DHP(r)/Cerveau,-,AGI-0074986,1E-05M,AGI-0074986_03_AGIOS_AGI-0074986,NaN,EASI_0002295353,2021-09-27,INHIB%,%,0.0,NaN
2,AGI-0074986_1E-05M,Optimized Security Profile,BB63EXT - 1,GABA Aa1b2g2,3051,GABA A(h)/[3H] Muscimol/CHO,Oncology,AGI-0074986,1E-05M,AGI-0074986_03_AGIOS_AGI-0074986,NaN,EASI_0002295422,2021-09-27,INHIB%,%,0.0,P18507
3,AGI-0074986_1E-05M,Optimized Security Profile,EPK511EXT - 2,LCK,2906,Lck(h)/LANCE,-,AGI-0074986,1E-05M,AGI-0074986_03_AGIOS_AGI-0074986,NaN,EASI_0002295413,2021-09-27,INHIB%,%,1.0,P06239
4,AGI-0074986_1E-05M,Optimized Security Profile,BY75EXT - 2,UP-DOPAMINE,52,Transporteur DA(h)/[3H] BTCP/CHO,Oncology,AGI-0074986,1E-05M,AGI-0074986_03_AGIOS_AGI-0074986,NaN,EASI_0002295314,2021-09-27,INHIB%,%,0.0,Q01959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13669,WEHI-1655870_1E-07M,Optimized Security Profile,EOR1EXT - 3,NO SYNTHASE C.,197,NOSc EDT(h),Metabolism Disease,WEHI-1655870,1E-07M,WEHI-1655870_02_WEHI_WEHI-1655870,NaN,EASI_0000649451,2020-10-16,INHIB%,%,0.0,P29474
13670,WEHI-1655870_1E-07M,Optimized Security Profile,EHE35EXT - 3,PDE IVD,4077,PDE IVD(h)/Sf9,Metabolism Disease,WEHI-1655870,1E-07M,WEHI-1655870_02_WEHI_WEHI-1655870,NaN,EASI_0000649517,2020-10-16,INHIB%,%,0.0,Q08499
13671,WEHI-1655870_1E-07M,Optimized Security Profile,BY75EXT - 2,UP-DOPAMINE,52,Transporteur DA(h)/[3H] BTCP/CHO,Metabolism Disease,WEHI-1655870,1E-07M,WEHI-1655870_02_WEHI_WEHI-1655870,NaN,EASI_0000649397,2020-10-16,INHIB%,%,0.0,Q01959
13672,WEHI-1655870_1E-07M,Optimized Security Profile,BY54EXT - 3,UP-5HT,439,Transporteur 5-HT(h)/[3H] Imipramine/CHO,Metabolism Disease,WEHI-1655870,1E-07M,WEHI-1655870_02_WEHI_WEHI-1655870,NaN,EASI_0000649463,2020-10-16,INHIB%,%,0.0,P31645


In [ ]:
# TEST!! ONCODESIGN ODS2015981_1E-05M

### About this part

About the Analysis:
1. Detected outliers. Initially we thought that results should be between 0 and 100, according to experts we kept the range between -50 and 150

About the dataframe
1. Sometimes we have only "Optimized Security Profile", we can also find only "Security Profile" and we can have both "Optimized Security Profile|Security Profile", the code allows to keep only 1 time each result, prioritizing the label "Optimized Security Profile"
2. It was found that we can have different number of experiments for one product because:
Sometimes we have same systeme etudie but diff code protocole and diff result, as an exemple we cab see Produit="S 80881-2_1e-07" with Systeme_Etudie="CANAL CA2+ TYP L"
In this case, according to expert's advise, we keep only the last result
3. By deleting duplicates, we can loose context about all the domains associated to one same experiment [Product+Studied system], so we concatenated all before deleting duplicates
